In [1]:
import matplotlib as mpl

import matplotlib.pyplot as plt
#import seaborn as sns
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

import networkx as nx

import pymysql
import pickle
import json

import numpy as np
from sklearn.decomposition import PCA

import time
import hdbscan
import umap

import spacy
from gensim.models import Word2Vec

import random
import re
from pmids2vec import pmids2vec, pmids2vec_titlesOnly
from pmids2corpus import pmids2corpus    #  todo integrate this into pmids2vec

### control parameters

In [2]:
path2clusteredPMIDs = 'data_processing_feb2020/pmids_1990_100k.json'
model_export_prefix = '1990_100k'
embed_titles_text = True
embed_abstracts_text = False


### get metadata

In [3]:
with open(path2clusteredPMIDs, 'r') as f:
    data = json.load(f)

N_samples = len(data['pmids'])

In [4]:
### embed abstract text

In [5]:
# todo force samples to have the same size, test impact of vocabulary ? 
#    or maybe just test variance over multiple samplesets to check for effects of sample size asymmetry

# load pmids

if embed_abstracts_text:

    for sample_id in range(N_samples):
        model_names = []

        with open(path2clusteredPMIDs, 'r') as f:
            data = json.load(f)

        #print(data['pmids'])   # i_sample, i_cluster
        sample_pmids = []

        clustered_pmids = data['pmids'][str(sample_id)]
        for cluster_id_str, pmids_list in clustered_pmids.items():

            model_export_path = 'data_processing_feb2020/abstracts_{}_cluster{}_{}.model'.format(
                                    sample_id, cluster_id_str, model_export_prefix)
            model_names.append(model_export_path)

            print('calling pmids2vec...')
            pmids2vec(pmids_list, model_export_path)
            
    print('--------------')
    

In [6]:
# recreate & save the corpus for good measure

if embed_abstracts_text:

    for sample_id in range(N_samples):
        model_names = []

        with open(path2clusteredPMIDs, 'r') as f:
            data = json.load(f)
        sample_pmids = []

        clustered_pmids = data['pmids'][str(sample_id)]
        for cluster_id_str, pmids_list in clustered_pmids.items():

            corpus_export_path = 'data_processing_feb2020/abstracts_{}_cluster{}_{}_corpus.json'.format(
                                    sample_id, cluster_id_str, model_export_prefix)
            model_names.append(corpus_export_path)

            print('calling pmids2corpus...')
            pmids2corpus(pmids_list, corpus_export_path)
            
    print('---------------')


In [7]:
### embed titles only

In [8]:
# titles only (saves the corpus while training the titles-only models)

# todo force samples to have the same size, test impact of vocabulary ? 
#    or maybe just test variance over multiple samplesets to check for effects of sample size asymmetry

# load pmids

if embed_titles_text:
    
    for sample_id in range(N_samples):
        model_names = []

                                 #  todo need much larger set of pmids since this is titles only
        with open(path2clusteredPMIDs, 'r') as f:
            data = json.load(f)

        #print(data['pmids'])   # i_sample, i_cluster
        sample_pmids = []
        clustered_pmids = data['pmids'][str(sample_id)]
        for cluster_id_str, pmids_list in clustered_pmids.items():

            model_export_path = 'data_processing_feb2020/titles_{}_cluster{}_{}'.format(
                    sample_id, cluster_id_str, model_export_prefix)

            print('calling pmids2vec...')
            pmids2vec_titlesOnly(pmids_list, model_export_path)
            
        print('---------------------------------')


calling pmids2vec...
SQL join executed in 338.06929993629456 s
SQL results fetched and cast in 0.07472109794616699 s
saving corpus of titles to data_processing_feb2020/titles_0_cluster0_1990_100k_titles__corpus.json
training word2vec model...
params: 20 dimensions, 5 window size, 20 min count
elapsed: 5.802192687988281
calling pmids2vec...
SQL join executed in 257.70791363716125 s
SQL results fetched and cast in 0.023283958435058594 s
saving corpus of titles to data_processing_feb2020/titles_0_cluster1_1990_100k_titles__corpus.json
training word2vec model...
params: 20 dimensions, 5 window size, 6 min count
elapsed: 2.9555301666259766
calling pmids2vec...
SQL join executed in 144.2593376636505 s
SQL results fetched and cast in 0.04142260551452637 s
saving corpus of titles to data_processing_feb2020/titles_0_cluster2_1990_100k_titles__corpus.json
training word2vec model...
params: 20 dimensions, 5 window size, 8 min count
elapsed: 2.1064319610595703
---------------------------------
cal